In [ ]:
import math
import pdb
import platform

import ai2thor
import copy
import time
import random
import ai2thor.controller
import datetime
import cv2
import os
import matplotlib

import matplotlib.pyplot as plt
import os
import numpy as np
# from utils.mid_level_constants import  scene_start_cheating_init_pose
import sys
from IPython.display import Image, display
import PIL.Image
sys.path.append('../')

from scripts.jupyter_helper import get_reachable_positions
from scripts.stretch_jupyter_helper import get_relative_stretch_current_arm_state,two_dict_equal, ARM_MOVE_CONSTANT, only_reset_scene, transport_wrapper, ADITIONAL_ARM_ARGS, execute_command, WRIST_ROTATION, get_current_wrist_state
from utils.stretch_utils.stretch_constants import STRETCH_ENV_ARGS, STRETCH_MANIPULATHOR_COMMIT_ID
from scripts.dataset_generation.find_categories_to_use import ROBOTHOR_SCENE_NAMES
matplotlib.use('TkAgg')
screen_size=224

STRETCH_ENV_ARGS['width'] = screen_size
STRETCH_ENV_ARGS['height'] = screen_size
STRETCH_ENV_ARGS['agentMode']='stretch'
STRETCH_ENV_ARGS['commit_id']=STRETCH_MANIPULATHOR_COMMIT_ID
STRETCH_ENV_ARGS['renderDepthImage'] = True
STRETCH_ENV_ARGS['renderInstanceSegmentation'] = True


In [ ]:

action_options = ['m', 'r', 'l', 'b', 'hu', 'hd', 'ao', 'ai', 'go', 'gc', 'wp', 'wn']

kitchens = [f"FloorPlan{i}" for i in range(1, 31)]
living_rooms = [f"FloorPlan{200 + i}" for i in range(1, 31)]
bedrooms = [f"FloorPlan{300 + i}" for i in range(1, 31)]
bathrooms = [f"FloorPlan{400 + i}" for i in range(1, 31)]

all_scenes = kitchens + living_rooms + bedrooms + bathrooms + ROBOTHOR_SCENE_NAMES
NUM_TESTS = 100
EPS_LEN = 500

def setup_thirdparty_camera(controller, camera_position):
    # controller.step('Pass')
    if len(controller.last_event.third_party_camera_frames) > 1:
        controller.step('UpdateThirdPartyCamera',
            thirdPartyCameraId=1, # id is available in the metadata response
            rotation=camera_position['rotation'],
            position=camera_position['position']
            )
    else:
        controller.step('AddThirdPartyCamera', 
            rotation=camera_position['rotation'], 
            position=camera_position['position'],
            fieldOfView=100)

def manual_task(controller, scene_name, logger_number =0, final=False, save_frames = False, init_sequence=[], verbose = False):
    only_reset_scene(controller, scene_name)
    all_actions = []
    all_action_details = []
    actions_ran_so_far = 0
    while(True):
        visualize(controller, save_frames)
        if len(init_sequence) > 0:
            action = init_sequence[0]
            init_sequence = init_sequence[1:]
        else:
            action = input('action?')
        if action == 'q':
            break
        all_actions.append(action)

        detail = execute_command(controller, action, ADITIONAL_ARM_ARGS)
        if verbose:
            print(detail, controller.last_event.metadata['lastActionSuccess'])
        if not controller.last_event.metadata['lastActionSuccess']:
            print('Failed', controller.last_event)
        all_action_details.append(detail)
        actions_ran_so_far += 1

        # controller.step(action='MoveArm', coordinateSpace="wrist", position=dict(x=0, y=0.2,z=0.2));visualize(controller, save_frames)
        # controller.step(action='RotateWristRelative', yaw=90);visualize(controller, save_frames)

    print(scene_name)
    print(all_actions)
    print(all_action_details)

def visualize(controller, save_frame=False, top_view=True):
    image = controller.last_event.frame
    arm_view = controller.last_event.third_party_camera_frames[0]
    combined=np.concatenate([image, arm_view], 1)
    if top_view:
        camera_position = {
            'position': controller.last_event.metadata['cameraPosition'],
            'rotation': dict(x=90,y=0,z=0)
        }
        camera_position['position']['y'] += 0.5
        setup_thirdparty_camera(controller, camera_position)
        third_view = controller.last_event.third_party_camera_frames[1]
        combined=np.concatenate([combined, third_view], 1)
            
    im = PIL.Image.fromarray(combined)
    display(im)
    
#     if STRETCH_ENV_ARGS['renderInstanceSegmentation']:
#         print(set([k.split('|')[0] for k in controller.last_event.instance_masks.keys()]))
#         print(set([k.split('|')[0] for k in controller.last_event.third_party_instance_masks[0].keys()]))
    
    if save_frame:
        saved_image_folder = '/Users/kianae/Desktop/saved_stretch_images_jupyter'
        os.makedirs(saved_image_folder, exist_ok=True)
        imagename = datetime.datetime.now().strftime("%m_%d_%Y_%H_%M_%S.%f")
        plt.imsave(os.path.join(saved_image_folder, imagename+'.png'), combined)
    



In [ ]:
controller = ai2thor.controller.Controller(**STRETCH_ENV_ARGS)

In [ ]:
# manual_task(controller, 'FloorPlan15', logger_number =0, init_sequence=['m', 'm', 'r', 'r', 'm', 'm', 'm', 'm', 'm', 'l', 'l', 'l', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', '/', 'zo', 'zo', 'zp', 'zp', 'ao', 'ao', 'ao', 'r', 'ao', 'p', 'hd', 'ao', 'ao', 'ao', 'ao', 'hd', 'hd', 'hd', '', 'hd', 'hd', 'ao', 'ao', 'wp', 'wn', 'wn', 'wn', 'wn', '/', 'p', 'u', 'u', 'hu', 'hu', 'r', 'r', 'r', 'r', 'r', 'm', 'm', 'm', 'd', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'r', 'r', 'l', 'ao', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'dh', 'dh', 'hd', 'hd', 'hd', 'hd', 'hd', 'dh', 'ao', 'ao', 'ao', 'ao', 'l', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'ao', 'ao', '/', 'p', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'd', 'r', 'r', 'r', 'r', 'm', 'm', 'm', 'm', 'm', 'm', '', 'mm', 'm', 'm', 'm', 'm', 'm', 'm', 'm'], save_frames=True, final=True)
manual_task(controller, 'FloorPlan15', logger_number =0, init_sequence=['m', 'm', 'r', 'r', 'm', 'm', 'm', 'm', 'm', 'l', 'l', 'l', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', '/', 'zo', 'zo', 'zp', 'zp', 'ao', 'ao', 'ao', 'r', 'ao', 'p', 'hd', 'ao', 'ao', 'ao', 'ao', 'hd', 'hd', 'hd', '', 'hd', 'hd', 'ao', 'ao', 'wp', 'wn', 'wn', 'wn', 'wn', '/', 'p', 'u', 'u', 'hu', 'hu', 'r', 'r', 'r', 'r', 'r', 'm', 'm', 'm', 'd', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'r', 'r', 'l', 'ao', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'dh', 'dh', 'hd', 'hd', 'hd', 'hd', 'hd', 'dh', 'ao', 'ao', 'ao', 'ao', 'l', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'ao', 'ao', '/', 'p', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'd', 'r', 'r', 'r', 'r', 'm', 'm', 'm', 'm', 'm', 'm', '', 'mm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'r', 'r', 'r', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'u', 'u', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'r', 'ao', 'p', 'u', 'u', 'hu', 'hu', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu'], save_frames=True, final=False)

In [ ]:
seq=['m', 'm', 'r', 'r', 'm', 'l', 'l', 'm', 'm', 'l', 'l', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'ao', 'ao', 'ao', 'ao', 'ao', 'hd', 'hd', 'hd', 'hd', 'dh', 'ao', 'ao', 'ao', 'hd', 'hd', 'dh', 'wp', 'wp', '/', 'p', 'hu', 'hu', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'wp', 'wp', 'wp', 'wp', 'wp', 'wp', 'l', 'l', 'l', 'l', 'l', 'l', 'r', 'r', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'r', 'm', 'm', 'r', 'm', 'm', 'm', 'm', 'm', 'l', 'm', 'm', 'l', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'r', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'l', 'l', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'd', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'hd', 'hd', 'hd', 'h', 'dh', 'hd', 'hd', 'hd', 'hd', 'dh', 'dh', 'dh', 'r', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'a', 'oao', 'hd', 'hd', 'dh', 'dh', 'l', 'hd', 'dh', 'dh', 'dh', 'ai', 'ai', 'ai', 'wn', 'wn', 'wn', 'wn', 'wn', 'wn', 'wn', 'wn', 'wn', 'wn', 'wn', 'wn', 'wn', 'wn', 'wn', 'ao', 'ao', 'hd', 'hd', 'hd', 'hd', '/', 'p', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'l', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'r', 'r', 'm', 'r', 'r', 'r', 'r', 'b', 'b', 'b', 'b', 'b', 'l', 'ao', 'ao', 'ao', 'ao', 'ao', 'a', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'd', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai']

manual_task(controller, ROBOTHOR_SCENE_NAMES[2], logger_number =0, init_sequence=seq,verbose=True,save_frames=False)

In [ ]:
controller.reset('FloorPlan301')
visualize(controller)

In [ ]:
seq = ['m', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'l', 'l', 'l', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'b', 'r', 'r', 'b', 'b', 'b', 'l', 'm', 'm', 'm', 'l', 'm', 'm', 'l', 'r', 'r', 'm', 'm', 'm', 'r', 'r', 'm', 'm', 'm', 'r', 'r', 'm', 'm', 'r', 'm', 'm', 'l', 'l', 'hu', 'hu', 'hu', 'hu', 'r', 'b', 'l', 'l', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'ao', 'hd', '/', 'p', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'l', 'b', 'b', 'r', 'b', 'b', 'b', 'b', 'b', 'b', 'b']

manual_task(controller, 'FloorPlan301', logger_number =0, init_sequence=seq, save_frames=True, final=False)

In [ ]:
print(controller.last_event.metadata['arm']['joints'][-1]['rootRelativePosition'])
controller.step('MoveArm', position=dict(x=0,y=.1,z=0))
print(controller.last_event.metadata['arm']['joints'][-1]['rootRelativePosition'])
controller.step('MoveArm', position=dict(x=0,y=1,z=0))
print(controller.last_event.metadata['arm']['joints'][-1]['rootRelativePosition'])
controller.step('MoveArm', position=dict(x=0,y=1,z=1))
print(controller.last_event.metadata['arm']['joints'][-1]['rootRelativePosition'])
controller.step('MoveArm', position=dict(x=0,y=2,z=0))
print(controller.last_event.metadata['arm']['joints'][-1]['rootRelativePosition'])
controller.step('MoveArm', position=dict(x=0,y=0,z=0.5))
print(controller.last_event.metadata['arm']['joints'][-1]['rootRelativePosition'])
controller.step('MoveArm', position=dict(x=0,y=0,z=0.0))
print(controller.last_event.metadata['arm']['joints'][-1]['rootRelativePosition'])

In [ ]:
for i in range(20):
    controller.step('MoveArm', position=dict(x=0,y=i*0.1,z=0.0))
    print(controller.last_event.metadata['arm']['joints'][-1]['rootRelativePosition'])
    
for i in range(20):
    controller.step('MoveArm', position=dict(x=0,y=0,z=i * 0.1))
    print(controller.last_event.metadata['arm']['joints'][-1]['rootRelativePosition'])
#     visualize(controller)

In [ ]:
visualize(controller)

In [ ]:
seq=['r', 'r', 'm', 'm', 'm', 'l', 'l', 'm', 'm', 'm', 'r', 'm', 'm', 'm', 'hu', 'hu', 'hu', 'hu', 'hu']
manual_task(controller, 'FloorPlan310', logger_number =0, init_sequence=seq, save_frames=False, final=False)

In [ ]:
controller.reset('FloorPlan310')
list_of_action = [{'action': 'PlaceObjectAtPoint', 'objectId': 'AlarmClock|+01.71|+00.67|-00.86', 'position': {'x': 1.039222240447998, 'y': 0.985424280166626, 'z': 0.23899996280670166}, 'forceKinematic': True}, {'action': 'AdvancePhysicsStep', 'simSeconds': 1.0}, {'action': 'PlaceObjectAtPoint', 'objectId': 'Book|-00.56|+00.84|-02.14', 'position': {'x': -1.2399827241897583, 'y': 1.9169363975524902, 'z': -2.0558547973632812}, 'forceKinematic': True}, {'action': 'AdvancePhysicsStep', 'simSeconds': 1.0}, {'action': 'TeleportFull', 'standing': True, 'x': -0.5, 'y': 0.9009925723075867, 'z': 0.0, 'rotation': {'x': 0, 'y': 180, 'z': 0}, 'horizon': 20}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': -0.2}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': -0.2}, {'action': 'RotateAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'degrees': -30}, {'action': 'RotateAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'degrees': -30}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'MoveArm', 'position': {'x': -7.213279604911804e-05, 'y': 0.05, 'z': -6.073713302612305e-05}, 'disableRendering': True, 'returnToStart': True, 'speed': 1}, {'action': 'MoveArm', 'position': {'x': -7.213279604911804e-05, 'y': 0.09999995231628418, 'z': -6.073713302612305e-05}, 'disableRendering': True, 'returnToStart': True, 'speed': 1}, {'action': 'MoveArm', 'position': {'x': -7.213279604911804e-05, 'y': 0.14999990463256835, 'z': -6.073713302612305e-05}, 'disableRendering': True, 'returnToStart': True, 'speed': 1}, {'action': 'MoveArm', 'position': {'x': -7.213279604911804e-05, 'y': 0.19999985694885253, 'z': -6.073713302612305e-05}, 'disableRendering': True, 'returnToStart': True, 'speed': 1}, {'action': 'MoveArm', 'position': {'x': -7.213279604911804e-05, 'y': 0.2499998092651367, 'z': -6.073713302612305e-05}, 'disableRendering': True, 'returnToStart': True, 'speed': 1}, {'action': 'MoveArm', 'position': {'x': -7.213279604911804e-05, 'y': 0.2999997615814209, 'z': -6.073713302612305e-05}, 'disableRendering': True, 'returnToStart': True, 'speed': 1}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}, {'action': 'RotateAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'degrees': 30}, {'action': 'MoveAgent', 'disableRendering': True, 'returnToStart': True, 'speed': 1, 'ahead': 0.2}]
for act in list_of_action:
    controller.step(**act)
    controller.step('Pass')
    visualize(controller)

In [ ]:
controller._build.url

In [ ]:
{'gridSize': 0.25, 'width': 224, 'height': 224, 'visibilityDistance': 1.5, 'fieldOfView': 100, 'agentControllerType': 'mid-level', 'server_class': <class 'ai2thor.fifo_server.FifoServer'>, 'useMassThreshold': True, 'massThreshold': 10, 'autoSimulation': False, 'autoSyncTransforms': True, 'renderInstanceSegmentation': True, 'agentMode': 'stretch', 'renderDepthImage': True, 'commit_id': 'cf23e657aa4738324d09cc79d5f78ea741bf20bf', 'x_display': '0.-1'}